# TabNet

In [ ]:
import os
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir(os.pardir)

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({'text.usetex': True,
                     'text.latex.preamble': r'\usepackage{amsmath}',
                     'font.family': 'serif'})
import numpy as np
#np.random.seed(0)
import pandas as pd
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.augmentations import RegressionSMOTE
import seaborn as sns
sns.set(style='ticks', font='serif')
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from timeit import default_timer as timer
import torch

## Data

In [ ]:
PATH_PROC_DATA = os.path.join('data', 'processed')
true_data = pd.read_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_clean.csv'))
syn_data = pd.read_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_synthetic.csv'))

features = ['d [mm]', 'f [GHz]', 'psPDtot_1 [W/m2]', 'psPDtot_4 [W/m2]']
target = 'pDeltaT * 100 [°C]'
X = syn_data[features].to_numpy()
y = syn_data[target].to_numpy() / 100

# train and validation set - synthetic data
X_train, X_valid, y_train, y_valid = train_test_split(X,
                                                      y[:, np.newaxis],
                                                      test_size=.2)

# test data - original data
X_test = true_data[features].to_numpy()
y_test = true_data[target].to_numpy() / 100

## Model

In [ ]:
reg = TabNetRegressor(n_d=32,
                      n_a=32,
                      n_steps=3,
                      gamma=1.3,
                      n_independent=3,
                      n_shared=3,
                      momentum=0.02,
                      lambda_sparse=0.001,
                      clip_value=1,
                      verbose=0,
                      optimizer_fn=torch.optim.Adam,
                      optimizer_params={'lr': 1e-3},
                      scheduler_fn=torch.optim.lr_scheduler.StepLR,
                      scheduler_params={'step_size': 500, 'gamma': 0.5},
                      mask_type='sparsemax')

In [ ]:
def train(save):
    aug = RegressionSMOTE(p=0.8)
    start = timer()
    reg.fit(X_train=X_train, y_train=y_train,
            eval_set=[(X_valid, y_valid)],
            eval_name=['test'],
            eval_metric=['mae', 'mse', 'rmse', 'rmsle'],
            max_epochs=500000,
            patience=1000,
            batch_size=128,
            virtual_batch_size=64,
            num_workers=4,
            drop_last=False,
            augmentations=aug)
    end = timer()
    elapsed = start - end
    print(f'Training finished in {elapsed:.2f} s.')
    save_path = reg.save_model(os.path.join('models', '05_tabnet'))
    print(f'Model saved ({save_path}).')
    return reg

In [ ]:
force_train = False
save = False

if force_train:
    print('Training the neural network...')
    reg = train(save)
else:
    try:
        print('Trying to restore the neural network...')
        reg.load_model(os.path.join('models', '05_tabnet.zip'))
        print('Restoring successful.')
    except Exception as e:
        print(e)
        print('Restoring failed. Training the neural network...')
        reg = train(save)

## Evaluating the predictive power

In [ ]:
y_pred = reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred.ravel()))

y_resid = (y_test - y_pred.ravel())
ae = np.abs(y_resid)
mae = np.mean(ae)
mae

In [ ]:
cs = sns.color_palette('rocket', 2)

fig, ax = plt.subplots(1, 1, figsize=(4.5, 4))
ax = sns.histplot(x=ae,
                  color=cs[0], stat='density', kde=True, ax=ax,
                  line_kws={'lw': 2})
ax.vlines(mae, *ax.get_ybound(), colors=cs[1], ls='--', lw=2,
          label='mean absolute error')
ax.legend()
ax.set(xlabel='', ylabel='',
       xticks=[0, mae, 0.25, 0.5],
       xticklabels=[0, round(mae, 2), 0.25, 0.5],
       xlim=[0, 0.5],
       yticks=[0, 11, 22],
       yticklabels=[0, 11, 22],
       ylim=[0, 22]
      )

fig.supxlabel('absolute error [°C]')
fig.supylabel('probability density')
fig.tight_layout()
sns.despine()

In [ ]:
PATH_ERROR_DATA = os.path.join('data', 'models')
error_data = os.path.join(PATH_ERROR_DATA, 'tabnet.npy')
save = False
if save:
    with open(error_data, 'wb') as f:
        np.save(f, ae)